In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image
from google.colab.patches import cv2_imshow
%tensorflow_version 1.x
import torchvision.models as models
import pandas as pd
import numpy as np

TensorFlow 1.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
foldername = '/content/drive/MyDrive/data_done/' 
train = pd.read_csv(foldername + 'my_csv.csv', names=['id','piece'])

In [6]:
np.random.seed(123)

def data_split(N, ratio=[8,2]):
  # generate a shuffle array
  shuffle_idx = np.arange(N)
  np.random.shuffle(shuffle_idx)
  # divide into train-val-test by the ratio
  data_split = (np.cumsum(ratio)/float(sum(ratio))*N).astype(int)
  out_idx = [None] * len(ratio)
  out_idx[0] = shuffle_idx[:data_split[0]]
  for i in range(1,len(ratio)):
    out_idx[i] = shuffle_idx[data_split[i-1] : data_split[i]]
  return out_idx  


#splitting into 8:2 ratio
split_idx = data_split(len(train))
df_train = train.loc[split_idx[0]]
df_valid = train.loc[split_idx[1]]

In [7]:
from torch.utils.data import Dataset
from PIL import Image

metadata_cols = train.columns[1:-1]
# make a child class of PyTorch's dataset class
class ChessDataset(Dataset):
    def __init__(self, root_dir, df, transforms=None):
        # initialization: called only once during creation
        self.root_dir = root_dir
        self.df = df
        self.file_names = df['id'].values
        self.targets = df['piece'].values
        self.transforms = transforms
        
    def __len__(self):
        # determine how many iterations in one epoch
        return len(self.df)
    
    def __getitem__(self, index):
        # called every time when the dataloader wants a sample
        # the dataset has a list of image file names
        # Input: dataloader provides a random index of the list
        # Output: corresponding image and meta data
        img_path = self.root_dir + self.file_names[index]
        img = Image.open(img_path)
        if self.transforms:
            img = self.transforms(img)
        target = self.targets[index]

        return img, target

In [22]:
transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    
    # hint: there are many "right" ways to do it
    # one idea is to take the center crop without randomflip, compared to transform_train
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [23]:
from torch.utils.data import DataLoader

train_dataset = ChessDataset(foldername, df_train, transforms=transform_train)
valid_dataset = ChessDataset(foldername, df_valid, transforms=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=8, shuffle=True) 

In [24]:
import torchvision.models as models
import torch
model = models.vgg16()
from collections import OrderedDict
classifier = torch.nn.Sequential(OrderedDict([('fc1', torch.nn.Linear(25088, 5000)),
                                        ('relu', torch.nn.ReLU()),
                                        ('drop', torch.nn.Dropout(p=0.5)),
                                        ('fc2', torch.nn.Linear(5000, 7)),
                                        ('output', torch.nn.LogSoftmax(dim=1))]))

model.classifier = classifier 
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [25]:
def validation(model, validateloader, criterion):
    
    val_loss = 0
    accuracy = 0
    
    for images, labels in iter(validateloader):

        images, labels = images.to('cpu'), labels.to('cpu')

        output = model.forward(images)
        val_loss += criterion(output, labels).item()

        probabilities = torch.exp(output)
        
        equality = (labels.data == probabilities.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return val_loss, accuracy

In [26]:
# Get GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(f'Using {device} device')

# Change input image depth for normalized images 
# model.conv1 = torch.nn.Conv2d(3, 8, kernel_size=(224, 224), stride=(2, 2), padding=(3, 3), bias=False)
criterion = torch.nn.NLLLoss()
#optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=.001, momentum=0.9)
model.to(device)

Using cpu device


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
def train_classifier():
        epochs = 15
        steps = 0
        print_every = 1

        model.to(device)

        for e in range(epochs):
        
            model.train()
    
            running_loss = 0
    
            for images, labels in iter(train_loader):
        
                steps += 1
        
                images, labels = images.to(device), labels.to(device)
        
                optimizer.zero_grad()
        
                output = model.forward(images)
                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()
        
                running_loss += loss.item()
        
                if steps % print_every == 0:
                
                    model.eval()
                
                    # Turn off gradients for validation, saves memory and computations
                    with torch.no_grad():
                        validation_loss, accuracy = validation(model, valid_loader, criterion)
            
                    print("Epoch: {}/{}.. ".format(e+1, epochs),
                          "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                          "Validation Loss: {:.3f}.. ".format(validation_loss/len(valid_loader)),
                          "Validation Accuracy: {:.3f}".format(accuracy/len(valid_loader)))
            
                    running_loss = 0
                    model.train()
                    
train_classifier()                    

Epoch: 1/15..  Training Loss: 1.944..  Validation Loss: 1.957..  Validation Accuracy: 0.188
Epoch: 1/15..  Training Loss: 1.947..  Validation Loss: 1.936..  Validation Accuracy: 0.287
Epoch: 1/15..  Training Loss: 1.897..  Validation Loss: 1.915..  Validation Accuracy: 0.287
Epoch: 1/15..  Training Loss: 1.886..  Validation Loss: 1.879..  Validation Accuracy: 0.325
Epoch: 1/15..  Training Loss: 1.798..  Validation Loss: 1.869..  Validation Accuracy: 0.325
Epoch: 1/15..  Training Loss: 1.719..  Validation Loss: 1.811..  Validation Accuracy: 0.325
Epoch: 1/15..  Training Loss: 2.027..  Validation Loss: 1.848..  Validation Accuracy: 0.287
Epoch: 2/15..  Training Loss: 1.524..  Validation Loss: 1.842..  Validation Accuracy: 0.287
Epoch: 2/15..  Training Loss: 1.614..  Validation Loss: 1.902..  Validation Accuracy: 0.287
Epoch: 2/15..  Training Loss: 1.864..  Validation Loss: 1.839..  Validation Accuracy: 0.325
Epoch: 2/15..  Training Loss: 1.540..  Validation Loss: 1.983..  Validation Accu

In [20]:
def test_accuracy(model, test_loader):

    # Do validation on the test set
    model.eval()
    model.to(device)

    with torch.no_grad():
    
        accuracy = 0
    
        for images, labels in iter(test_loader):
    
            images, labels = images.to(device), labels.to(device)
    
            output = model.forward(images)

            probabilities = torch.exp(output)
        
            equality = (labels.data == probabilities.max(dim=1)[1])
        
            accuracy += equality.type(torch.FloatTensor).mean()
        
        print("Test Accuracy: {}".format(accuracy/len(valid_loader)))    
        
        
test_accuracy(model, valid_loader)

Test Accuracy: 0.2874999940395355


In [ ]:

it = 0
for i in range(1):
  # Train the model:
  for batch_idx, (data, target) in enumerate(train_loader):      
      print(f"batch iteration={it}")
      data, target = data.to(device), target.to(device)
      target = target.float()
      optimizer.zero_grad()
      predictions = model(data)
      real_predictions = []
      for row in predictions:
        guess = torch.argmax(torch.nn.functional.softmax(row)).tolist()
        real_predictions.append(guess)
      real_predictions = torch.tensor(real_predictions)
      real_predictions = real_predictions.float()
      loss = lossf(real_predictions, target)
      loss.backward()
      optimizer.step()
      it+=1